# VIH Noisy Dynamic Identification

In [1]:
using DrWatson
@quickactivate :ExtendedKalmanFilterNeuralTraining
#@quickactivate "ExtendedKalmanFilterNeuralTraining"; include(srcdir("ExtendedKalmanFilterNeuralTraining.jl"))

using XLSX, DataFrames

In [2]:
using ModelingToolkit, DifferentialEquations

@named hiv = HIV()
display(hiv)

HIV_tspan = (0.0, 365.0 * 10)
HIV_prob = ODEProblem(hiv, [], HIV_tspan)
HIV_sol = solve(HIV_prob);

x = DataFrame(hcat(map(HIV_sol, 0.0:2:max(HIV_tspan...))...)', ["T", "T_inf", "M", "M_inf", "V"]);

Model hiv with 5 equations
States (5):
  T(t) [defaults to 1000.0]
  T_inf(t) [defaults to 0.0]
  M(t) [defaults to 150.0]
  M_inf(t) [defaults to 0.0]
⋮
Parameters (15):
  ρ_T [defaults to 0.01]
  C_T [defaults to 300]
  k_T [defaults to 4.57e-5]
  s_T [defaults to 10.0]
⋮

### EKF and Tikonov EKF algorithm comparison

In [6]:
allparams = Dict(
    "N" => 50, # it is inside vector. It is expanded.
    "dB" => 20.0,
    "alg" => ["EKF!", "TikonovEKF!"],     # single element inside vector; no expansion
    "noise" => "gaussian_noise", # not in vector = not expanded, even if naturally iterable
)

dicts = dict_list(allparams)
for d in dicts
    f = makesim(d, x)
    wsave(datadir("EFKvsTikonovEFK", savename(d, "jld2")), f)
end

EFKvsTikonovEFK = collect_results(datadir("EFKvsTikonovEFK"))

XLSX.writetable(datadir("EFKvsTikonovEFK", "full_table.xlsx"), EFKvsTikonovEFK)

┌ Info: Scanning folder c:\Users\Daniel I Parra Verde\OneDrive\Documentos\Proyectos\ExtendedKalmanFilterNeuralTraining\data\EFKvsTikonovEFK for result files.
└ @ DrWatson C:\Users\Daniel I Parra Verde\.julia\packages\DrWatson\S4dYA\src\result_collection.jl:115
┌ Info: Added 2 entries.
└ @ DrWatson C:\Users\Daniel I Parra Verde\.julia\packages\DrWatson\S4dYA\src\result_collection.jl:191


### TikonovEFK robustness of gaussian noise
 

In [7]:
allparams = Dict(
    "N" => 5, # it is inside vector. It is expanded.
    "dB" => collect(0:1:20),
    "alg" => "TikonovEKF!",     # single element inside vector; no expansion
    "noise" => "gaussian_noise", # not in vector = not expanded, even if naturally iterable
)

dicts = dict_list(allparams)
for d in dicts
    f = makesim(d, x)
    wsave(datadir("Robustness", savename(d, "jld2")), f)
end

Robustness = collect_results(datadir("Robustness"))

XLSX.writetable(datadir("Robustness", "full_table.xlsx"), Robustness)

┌ Info: Scanning folder c:\Users\Daniel I Parra Verde\OneDrive\Documentos\Proyectos\ExtendedKalmanFilterNeuralTraining\data\Robustness for result files.
└ @ DrWatson C:\Users\Daniel I Parra Verde\.julia\packages\DrWatson\S4dYA\src\result_collection.jl:115
┌ Info: Added 21 entries.
└ @ DrWatson C:\Users\Daniel I Parra Verde\.julia\packages\DrWatson\S4dYA\src\result_collection.jl:191


### TikonovEFK robustness of noise with different distributions

In [8]:
allparams = Dict(
    "N" => 5, 
    "dB" => collect(0:1:20),
    "alg" => "TikonovEKF!",     
    "noise" => ["gaussian_noise", "uniform_noise"]
)

dicts = dict_list(allparams)
for d in dicts
    f = makesim(d, x)
    wsave(datadir("NoiseTypeComparison", savename(d, "jld2")), f)
end

NoiseTypeComparison = collect_results(datadir("NoiseTypeComparison"))

XLSX.writetable(datadir("NoiseTypeComparison", "full_table.xlsx"), NoiseTypeComparison)